#### Imports ans installations

In [0]:
%sh pip install keras

In [0]:
%sh pip install handyspark

In [0]:
%sh pip install mlflow

In [0]:
%sh pip install tensorflow

In [0]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
import time
#from tensorflow.keras.layers import Dense, Flatten, Conv2D
#from tensorflow.keras import Model
import os
import tempfile
import keras
import numpy as np
import random
import pandas as pd
from pyspark.sql.functions import trim
pd.set_option('max_colwidth', -1) # to prevent truncating of columns in jupyter
#from six.moves import urllib
#from pyspark.ml.functions import vector_to_array
# import libraries
import os
from sklearn.preprocessing import MinMaxScaler
#from sklearn.externals import joblib
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
%matplotlib inline

from numpy.random import seed
import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR)
from multiprocessing.pool import ThreadPool

from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers
# set random seed
seed(10)
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K

from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from pyspark.ml.pipeline import PipelineModel
import pyspark.sql.functions as F
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.sql.functions import udf
from pyspark.sql.functions import col
from pyspark.sql.functions import rand
from pyspark.sql.types import FloatType
import mlflow
import mlflow.mleap
import mlflow.spark
import time
from pyspark.sql import SparkSession
from handyspark import *
import findspark
from datetime import datetime, timedelta
import mleap.pyspark
from mleap.pyspark.spark_support import SimpleSparkSerializer
import random
import string
from functools import reduce
from pyspark.sql import DataFrame

#### Modeling

In [0]:
def one_step_attention(a, s_prev,repeator, concatenator,densor1, densor2,activator, dotor):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attention) LSTM cell
    """
    
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a"
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis 
    concat = concatenator([a,s_prev])
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. 
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. 
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" 
    alphas = activator(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell.
    context = dotor([alphas,a])
    
    return context
  
  
def sequence_transcation_fraud_detection_combined_with_entity_embeddings(Tx, Ty, n_a, n_s, input_size, out_size,index_positions_dict,index_original_dimensions_dict, index_desired_dimensions_dict,cat_features_amount):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM

    Returns:
    model -- Keras model instance
    """
    
    #Define the components of the attention mechanism of which the weights are going to be shared acorss time steps. 
    repeator = RepeatVector(Tx)
    concatenator = Concatenate(axis=-1)
    densor1 = Dense(10, activation = "tanh")
    densor2 = Dense(1, activation = "relu")
    activator = Activation("softmax", name='attention_weights') 
    dotor = Dot(axes = 1)

    
    # Define the post_attention_lstm layer and the dense layers for the final hidden state of the post_attention_lstm layer.
    post_activation_LSTM_cell = LSTM(n_s, return_state = True) # post-attention LSTM 
    output_layer_1 = Dense(16, activation="relu")
    output_layer_2 = Dense(16, activation="relu")
    output_layer_3 = Dense(16, activation="relu")
    output_layer_4 = Dense(8, activation="relu")
    output_layer_5 = Dense(1, activation="sigmoid")
    
    bn = keras.layers.BatchNormalization()
    drop = keras.layers.Dropout(0.5)
    
    
    # Define the entity embeddings component for the categorical variables
    embeddings_transform_layers = {}
    embeddings_bn_layers = {}
    embeddings_reshape_layers = {}
    features_trans_layers = {}
    features_trans_bn_layers = {}
    
            
    for key in index_positions_dict.keys():

      embeddings_transform_layer_name = key + "embedding_transform"
      embeddings_bn_layer_name = key + "embedding_bn"
      embeddings_reshape_layer_name = key + "embedding_reshape"
      features_trans_layer_name = key + "features_trans"
      features_trans_bn_layer_name = key + "features_trans_bn"
      

      index_original_dimensions= index_original_dimensions_dict[key]
      index_desired_dimensions = index_desired_dimensions_dict[key]

      #Embeddings
      embeddings_transform_layer = tf.keras.layers.Embedding(index_original_dimensions,index_desired_dimensions)
      embeddings_reshape_layer = tf.keras.layers.Reshape((Tx,index_desired_dimensions, ))
      embeddings_bn_layer = tf.keras.layers.BatchNormalization()
      
      #Transformation for the embeddings
      features_trans_layer = tf.keras.layers.Dense(index_desired_dimensions, activation="relu")
      features_trans_bn_layer = tf.keras.layers.BatchNormalization()
      
      #Add the layers to cooresponding dicts
      embeddings_transform_layers.update({embeddings_transform_layer_name : embeddings_transform_layer})
      embeddings_reshape_layers.update({embeddings_reshape_layer_name : embeddings_reshape_layer})
      embeddings_bn_layers.update({embeddings_bn_layer_name : embeddings_bn_layer})
      
      features_trans_layers.update({features_trans_layer_name:features_trans_layer})
      features_trans_bn_layers.update({features_trans_bn_layer_name:features_trans_bn_layer})
    
    
    # Define the inputs of the model with a shape (Tx,)
    # Define s0 (initial hidden state) and c0 (initial cell state)
    # for the decoder LSTM with shape (n_s,)
    X = Input(shape=(Tx, input_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
 
    cat_embedding_features_list = []
    one_hot_features_list = []
    
    #generate entity embeddings for each categorical variable in a for loop 
    #i = 0
    for key in index_positions_dict.keys():

      i  = index_positions_dict[key]
      index_feature = X[:,:,i]

      index_feature =  tf.cast(index_feature, tf.int32)
    
      index_feature_int =  tf.cast(index_feature, tf.int32)
      index_feauture_one_hot = tf.one_hot(index_feature_int, index_original_dimensions)

      index_feature = embeddings_transform_layers[key + "embedding_transform"](index_feature)
      index_feature = embeddings_reshape_layers[key + "embedding_reshape"](index_feature)
      index_feature = embeddings_bn_layers[key + "embedding_bn"](index_feature)
      
      index_feature = features_trans_layers[key + "features_trans"](index_feature)
      index_feature = features_trans_bn_layers[key + "features_trans_bn"](index_feature)
      
      
      cat_embedding_features_list.append(index_feature)
      one_hot_features_list.append(index_feauture_one_hot)

    # concatenate the entity embeddings with the numeric features 
    numeric_features = X[:,:,max(len(index_positions_dict.keys())-1,cat_features_amount):]
    features_list = cat_embedding_features_list

    features_list.append(numeric_features)                    
    features = tf.keras.layers.concatenate(features_list)
    cat_embedding_features = tf.keras.layers.concatenate(cat_embedding_features_list)
    cat_one_hot_features = tf.keras.layers.concatenate(one_hot_features_list)
    
    
    a = Bidirectional(LSTM(units=n_a, return_sequences=True))(numeric_features)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
        
        context = one_step_attention(a, s, repeator = repeator, concatenator = concatenator,densor1 = densor1, densor2 = densor2,activator = activator, dotor = dotor )
        
        s, _, c = post_activation_LSTM_cell(inputs=context, initial_state=[s, c])
        
        out = output_layer_1(inputs=s)
        out = bn(inputs=out)
        out = output_layer_2(inputs=out)
        #out = self.drop(inputs=out)
        #out = self.output_layer_3(inputs=out)
        #out = self.output_layer_4(inputs=out)
        final_out = output_layer_5(inputs=out)
        
        # Step 2.D: Append "out" to the "outputs" list
        outputs.append(final_out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. 
    model = Model(inputs=[X,s0,c0], outputs=outputs)
       
    return model 
  
  
def sequence_transcation_fraud_detection(Tx, Ty, n_a, n_s, input_size, out_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM

    Returns:
    model -- Keras model instance
    """
    
    repeator = RepeatVector(Tx)
    concatenator = Concatenate(axis=-1)
    densor1 = Dense(10, activation = "tanh")
    densor2 = Dense(1, activation = "relu")
    activator = Activation("softmax", name='attention_weights') 
    dotor = Dot(axes = 1)

    
    post_activation_LSTM_cell = LSTM(n_s, return_state = True) # post-attention LSTM 
    output_layer_1 = Dense(16, activation="relu")
    output_layer_2 = Dense(16, activation="relu")
    output_layer_3 = Dense(16, activation="relu")
    output_layer_4 = Dense(8, activation="relu")
    output_layer_5 = Dense(1, activation="sigmoid")
    bn = keras.layers.BatchNormalization()
    drop = keras.layers.Dropout(0.5)
    
    # Define the inputs of the model with a shape (Tx,)
    # Define s0 (initial hidden state) and c0 (initial cell state)
    # for the decoder LSTM with shape (n_s,)
    X = Input(shape=(Tx, input_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
   
    a = Bidirectional(LSTM(units=n_a, return_sequences=True))(X)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
        
        context = one_step_attention(a, s, repeator = repeator, concatenator = concatenator,densor1 = densor1, densor2 = densor2,activator = activator, dotor = dotor )
        
        s, _, c = post_activation_LSTM_cell(inputs=context, initial_state=[s, c])
        
        out = output_layer_1(inputs=s)
        out = bn(inputs=out)
        out = output_layer_2(inputs=out)
        #out = self.drop(inputs=out)
        #out = self.output_layer_3(inputs=out)
        #out = self.output_layer_4(inputs=out)
        final_out = output_layer_5(inputs=out)
        
        # Step 2.D: Append "out" to the "outputs" list
        outputs.append(final_out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. 
    model = Model(inputs=[X,s0,c0], outputs=outputs)
       
    return model

#### Load in the data sets

In [0]:
#04 -- 08

label_col  = "doublelabel"
customer = "none"
start_test = "2019-12-01"
end_test = "2019-12-31"
end_train = "2019-08-31"

train_path = '/tuser/hive/warehouse/xinghao/tensorflow/ohe/seq_ee_viva_cards_train_1904_1908_sample_rate_004.tfrecord'
test_path = '/tuser/hive/warehouse/xinghao/tensorflow/ohe/seq_ee_viva_cards_test_1912_full.tfrecord'



train= spark.table("xinghao.seq_ee_viva_cards_train_1904_1908_sample_rate_004")

test = spark.table("xinghao.seq_ee_viva_cards_test_1912_full")

features_size = 471 

train_size = train.count()
test_size = test.count()


test_label_table  = test.filter((test.row_number % 2) == 0).select(["authtimestamp","doublelabel"])
train_label_table  = train.filter((train.row_number % 2) == 0).select(["authtimestamp","doublelabel"])

#### Decode the tf_records for training and testing

In [0]:
#Dict first only of float32, int64, string
def decode(serialized_example, features_dict=None):
  """
  Parses an image and label from the given `serialized_example`.
  It is used as a map function for `dataset.map`
  """
  if features_dict is None:
    features_dict = {
          'features': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
          'doublelabel': tf.io.FixedLenFeature([], tf.float32),
      }
  
 # 1. define a parser
  features = tf.io.parse_single_example(
      serialized_example,
      # Defaults are not specified since both keys are required.
      features=features_dict)

  # 2. Convert the data
  feats = features['features']
  label = features['doublelabel']
  
  return feats, label



filenames = [("/dbfs" + train_path + "/" + name) for name in os.listdir("/dbfs" + train_path) if name.startswith("part")]
train_dataset_exp = tf.data.TFRecordDataset(filenames)
train_dataset_exp = train_dataset_exp.map(decode)


filenames = [("/dbfs" + test_path + "/" + name) for name in os.listdir("/dbfs" + test_path) if name.startswith("part")]
test_dataset_exp = tf.data.TFRecordDataset(filenames)
test_dataset_exp = test_dataset_exp.map(decode)

#### Extract features and labels from training and testing sets

In [0]:
for batch in train_dataset_exp.batch(train_size).take(1):
  train_features = batch[0]
  train_labels = batch[1] 

  
for batch in test_dataset_exp.batch(test_size).take(1):
  test_features = batch[0]
  test_labels = batch[1] 
  #numerical = batch[2]
  #print(autoencoder_model(features,training= False))
  print(test_features)
  print(test_labels)

#### Reshape the features and labels for training and testing

In [0]:
train_fea = tf.reshape(train_features,[int(train_size/2),2,features_size])
#valid_fea = tf.reshape(valid_features,[int(validation_size/2),2,features_size])
test_fea = tf.reshape(test_features,[int(test_size/2),2,features_size])
#test_lab = tf.reshape(test_labels,[30267,1,2])

train_label = tf.reshape(train_labels,[int(train_size/2),2,1])
label = tf.transpose(train_label, [1, 0, 2])

#### Training

In [0]:
from keras.optimizers import SGD
opt = SGD(lr=0.01)

#opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
#model.compile(loss = "binary_crossentropy", optimizer = opt,metrics=["accuracy"])
#model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])

METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

EPOCHS = 200
BATCH_SIZE = 2048

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor= "val_dense_6_1_auc", 
    verbose=1,
    patience=20,
    mode='max',
    restore_best_weights=True)



index_positions_dict = {"mcccode":0,"issuingbank":1,"merchant":2,"cardcommercial":3,"cardcountry":4}

index_original_dimensions_dict =  {"mcccode":244,"issuingbank":2618,"merchant":16673,"cardcommercial":2,"cardcountry":142}

index_desired_dimensions_dict = {"mcccode":32,"issuingbank":48,"merchant":32,"cardcommercial":2,"cardcountry":16}

model = sequence_transcation_fraud_detection_combined_with_entity_embeddings(2, 2, 64, 64, 471, 1, index_positions_dict, index_original_dimensions_dict,index_desired_dimensions_dict, 5 )

n_a = 64 # number of units for the pre-attention, bi-directional LSTM's hidden state 'a'
n_s = 64 # number of units for the post-attention LSTM's hidden state "s"

  
s0 = tf.zeros([int(train_size/2), n_s], tf.int32)
c0 = tf.zeros([int(train_size/2), n_s], tf.int32)
  

lr = tf.keras.optimizers.schedules.ExponentialDecay(0.001, decay_steps=100, decay_rate=0.9, staircase=False)
optimizer = tf.keras.optimizers.Adam(lr)

# keras.optimizers.Adam(lr=1e-4)

model.compile(
      optimizer= keras.optimizers.Adam(lr=1e-4),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=METRICS)

In [0]:
tf.keras.backend.clear_session() 
model.fit([train_fea,s0, c0], list(label), epochs=200,callbacks = [early_stopping], batch_size=1000, validation_split=0.2)

#### Results

In [0]:
firstelement=udf(lambda v:float(v[0]),FloatType())
secondelement=udf(lambda v:float(v[1]),FloatType())

s1 = tf.zeros([int(test_size/2), n_s], tf.int32)
c1 = tf.zeros([int(test_size/2), n_s], tf.int32)

#s1 = np.zeros((int(test_size),n_s))
#c1 = np.zeros((int(test_size),n_s))

test_pred = model.predict([test_fea,s1, c1])
train_pred = model.predict([train_fea,s0, c0])

train_pred = pd.DataFrame(train_pred[1])
test_pred = pd.DataFrame(test_pred[1])

train_pred[1] = train_pred[0]
train_pred[0] = 1- train_pred[0]
test_pred[1] = test_pred[0]
test_pred[0] = 1- test_pred[0]

#test_label = pd.DataFrame(test_label_table2)
test_label = test_label_table.toPandas()
#test_label = test_label.rename(columns={0: "authtimestamp",1:"doublelabel"})
test_label = test_label.rename(columns={"cb_flag":"doublelabel"})


#train_label = pd.DataFrame(train_label_table)
train_label = train_label_table.toPandas()
#train_label = train_label.rename(columns={0: "authtimestamp",1:"doublelabel"})
train_label = train_label.rename(columns={"cb_flag":"doublelabel"})

result_test = pd.concat([test_label, test_pred], axis=1).reindex(test_pred.index)
result_train = pd.concat([train_label, train_pred], axis=1).reindex(train_pred.index)

result_test = spark.createDataFrame(result_test)

from pyspark.sql.functions import when
result_test = result_test.withColumn("predictions", \
              when((result_test["0"] > 0.8) , 0).otherwise(1))
result_test = result_test.withColumn("minus_pred", -1*result_test["0"])

result_train= spark.createDataFrame(result_train)

from pyspark.sql.functions import when
result_train = result_train.withColumn("predictions", \
              when((result_train["0"] > 0.8) , 0).otherwise(1))
result_train = result_train.withColumn("minus_pred", -1*result_train["0"])

result_test=VectorAssembler(inputCols=["0","minus_pred"], outputCol="rawPrediction").transform(result_test)  
result_test=VectorAssembler(inputCols=["0","1"], outputCol="probability").transform(result_test)

result_train=VectorAssembler(inputCols=["0","minus_pred"], outputCol="rawPrediction").transform(result_train)  
result_train=VectorAssembler(inputCols=["0","1"], outputCol="probability").transform(result_train)

result_test = result_test.drop("minus_pred")
result_train = result_train.drop("minus_pred")

In [0]:
train_pred=result_train.withColumn("prob1",firstelement(F.col("probability")))\
                     .withColumn("prob2",secondelement(F.col("probability")))\
                     .withColumn("rawpred1",firstelement(F.col("rawprediction")))\
                     .withColumn("rawpred2",secondelement(F.col("rawprediction")))


test_pred=result_test.withColumn("prob1",firstelement(F.col("probability")))\
                   .withColumn("prob2",secondelement(F.col("probability")))\
                   .withColumn("rawpred1",firstelement(F.col("rawprediction")))\
                   .withColumn("rawpred2",secondelement(F.col("rawprediction")))

In [0]:
def show_null_count(df):
  #display([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns])
  display(df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]))

In [0]:
def getscoreswithrecall(predbythreshold,TPRgoal):
  predfilter = predbythreshold.where(predbythreshold.recall <=TPRgoal)
  tpr = predfilter.agg({"recall": "max"}).head()[0]
  fpr = predfilter.agg({"fpr": "max"}).head()[0]
  threshold = predfilter.agg({"threshold": "min"}).head()[0]
  f.write("FPR: "+ str(round(fpr,3))+", TPR: "+ str(round(tpr,3))+", Threshold: "+str(round(threshold,3))+"\n")
  return fpr,tpr,threshold

In [0]:
#Scores test set
handypredictions = test_pred.toHandy()
bcm = BinaryClassificationMetrics(handypredictions, scoreCol='probability', labelCol=label_col)
predbythreshold_overall=bcm.getMetricsByThreshold()
fpr01,tpr01,threshold01 = reusable_functions.getscoreswithfpr(predbythreshold_overall,0.1)
fpr001,tpr001,threshold001 = reusable_functions.getscoreswithfpr(predbythreshold_overall,0.01)
fpr0001,tpr0001,threshold0001 = reusable_functions.getscoreswithfpr(predbythreshold_overall,0.001)

if customer != "none":
  #Scores unseen test set
  handypredictions = unseen_test_pred.toHandy()
  bcm = BinaryClassificationMetrics(handypredictions, scoreCol='probability', labelCol=label_col)
  predbythreshold_unseen=bcm.getMetricsByThreshold()
  unseen_fpr01,unseen_tpr01,unseen_threshold01 = reusable_functions.getscoreswithfpr(predbythreshold_unseen,0.1)
  unseen_fpr001,unseen_tpr001,unseen_threshold001 = reusable_functions.getscoreswithfpr(predbythreshold_unseen,0.01)
  unseen_fpr0001,unseen_tpr0001,unseen_threshold0001 = reusable_functions.getscoreswithfpr(predbythreshold_unseen,0.001)

#Scores start week
start_week=start_test
end_week=datetime.strftime(datetime.strptime(start_week, "%Y-%m-%d") + timedelta(days=7), "%Y-%m-%d")
week_pred=handypredictions.filter(col("authtimestamp")>=start_week).filter(col("authtimestamp")<end_week)
predbythreshold_start = BinaryClassificationMetrics(week_pred, scoreCol='probability', labelCol=label_col).getMetricsByThreshold()
fpr,tpr01_start,threshold = reusable_functions.getscoreswithfpr(predbythreshold_start,0.1)
fpr,tpr001_start,threshold = reusable_functions.getscoreswithfpr(predbythreshold_start,0.01)
fpr,tpr0001_start,threshold = reusable_functions.getscoreswithfpr(predbythreshold_start,0.001)

#Scores end week
start_week=datetime.strftime(datetime.strptime(end_test, "%Y-%m-%d") - timedelta(days=7), "%Y-%m-%d")
end_week=end_test
week_pred=handypredictions.filter(col("authtimestamp")>=start_week).filter(col("authtimestamp")<end_week)
predbythreshold_end = BinaryClassificationMetrics(week_pred, scoreCol='probability', labelCol=label_col).getMetricsByThreshold()
fpr,tpr01_end,threshold = reusable_functions.getscoreswithfpr(predbythreshold_end,0.1)
fpr,tpr001_end,threshold = reusable_functions.getscoreswithfpr(predbythreshold_end,0.01)
fpr,tpr0001_end,threshold = reusable_functions.getscoreswithfpr(predbythreshold_end,0.001)

#Decrease in score
if tpr01_start==0:
  perfdecr_01=0
else:
  perfdecr_01=(tpr01_end-tpr01_start)/tpr01_start

if tpr001_start==0:
  perfdecr_001=0
else:
  perfdecr_001=(tpr001_end-tpr001_start)/tpr001_start

if tpr0001_start==0:
  perfdecr_0001=0
else:
  perfdecr_0001=(tpr0001_end-tpr0001_start)/tpr0001_start

In [0]:
bcm.plot_roc_curve()

In [0]:
import math
from multiprocessing.pool import ThreadPool

def mcc_score_per_threshold(predictions, label_col, threshold, verbose=False):
  predictions= predictions.withColumn("prediction",F.when(col("prob2")>threshold,1.0).otherwise(0.0))
#   print(predictions.head())
  tp= predictions.filter((col(label_col) == 1) & (col("prediction") == 1)).count()
  tn= predictions.filter((col(label_col) == 0) & (col("prediction") == 0)).count()
  fp= predictions.filter((col(label_col) == 0) & (col("prediction") == 1)).count()
  fn= predictions.filter((col(label_col) == 1) & (col("prediction") == 0)).count()
  
  try:
    mcc = (tp * tn - fp * fn) / math.sqrt((tp + fp) * (tp + fn) * (fp + tn) * (tn + fn))
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
  except:
    print("Exception raised for threshold {}. Setting mcc score to 0. Partial results: True Positives {}, True Negatives {}, False Positives {}, False Negatives {}".format(threshold, tp, tn, fp, fn))
    mcc = 0
    recall = 0
    precision = 0
  
  if verbose:
      print("Threshold: {}, Mcc: {}, Recall: {}. True Positives {}, True Negatives {}, False Positives {}, False Negatives {}".format(threshold, mcc, recall, tp, tn, fp, fn))
  return mcc, threshold, recall, precision

def max_mcc_score(predictions, label_col, parallel=False, min_recall=0.25):
  
  import numpy as np
  listOfThresholds = np.arange(0, 1, 0.02).tolist()
  
  if parallel:
    pool = ThreadPool(10)
    scores = pool.map(lambda threshold: mcc_score_per_threshold(predictions, label_col, threshold), listOfThresholds)
  else:
    scores = [mcc_score_per_threshold(predictions, label_col, threshold) for threshold in listOfThresholds]
    
  # get mcc scores with recall > min_recall
  scores_min_recall = [mcc if recall>min_recall else 0 for mcc, threshold, recall, precision in scores]
  max_mcc_idx=np.argmax(scores_min_recall)
  return scores[max_mcc_idx] # mcc, threshold, recall, precision

In [0]:
#max_mcc_tuple=reusable_functions.max_mcc_score(test_pred, label_col, True, 0.25) # mcc, threshold, recall, precision

max_mcc_tuple=max_mcc_score(test_pred, label_col, True, 0.25)

def print_cm_by_thresh(predictions,threshold, print_cm = True):
  predictions= predictions.withColumn("prediction",F.when(col("prob2")>threshold,1.0).otherwise(0.0))
  tp= predictions.filter((col(label_col) == 1) & (col("prediction") == 1)).count()
  tn= predictions.filter((col(label_col) == 0) & (col("prediction") == 0)).count()
  fp= predictions.filter((col(label_col) == 0) & (col("prediction") == 1)).count()
  fn= predictions.filter((col(label_col) == 1) & (col("prediction") == 0)).count()
  if print_cm:
    print("TP: ", tp)
    print("FP: ", fp)
    print("TN: ", tn)
    print("FN: ", fn)
  return [tp,tn,fp,fn]

max_mcc_tp, max_mcc_tn, max_mcc_fp, max_mcc_fn = print_cm_by_thresh(test_pred, max_mcc_tuple[1], print_cm = False)

precision_sum = max_mcc_tp + max_mcc_fp
recall_sum = max_mcc_tp + max_mcc_fn
if (max_mcc_tp == 0) or (precision_sum == 0) or (recall_sum == 0):
  max_mcc_precision = 0
  max_mcc_recall = 0
else:
  max_mcc_precision = max_mcc_tp / (precision_sum)
  max_mcc_recall = max_mcc_tp / (recall_sum)

In [0]:
def green_score(predictions,threshold):
  predictions = predictions.withColumn("prediction",F.when(col("prob2")>threshold,1.0).otherwise(0.0))
  tp= predictions.filter((col(label_col) == 1) & (col("prediction") == 1)).count()
  tn= predictions.filter((col(label_col) == 0) & (col("prediction") == 0)).count()
  fp= predictions.filter((col(label_col) == 0) & (col("prediction") == 1)).count()
  fn= predictions.filter((col(label_col) == 1) & (col("prediction") == 0)).count()
  try:
    tnr = tn / (tn+fp)
    recall = tp / (tp+fn)
    avg_green = (tnr + recall) / 2
    return [avg_green, tnr, recall]
  except:
    return 0
  
listOfThresholds = np.arange(0.01,0.2,0.04) #np.arange(0,0.2,0.01) #[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
pool = ThreadPool(5)

green_results = pool.map(lambda threshold: green_score(test_pred,threshold), listOfThresholds)
for i, thresh in enumerate(listOfThresholds):
  green_results[i].append(thresh)
  green_info = max(green_results)

In [0]:
#Plot ROC-Curve
class CurveMetrics(BinaryClassificationMetrics):
    def __init__(self, *args):
        super(CurveMetrics, self).__init__(*args)

    def _to_list(self, rdd):
        points = []
        for row in rdd.collect():
            points += [(float(row._1()), float(row._2()))]
        return points

    def get_curve(self, method):
        rdd = getattr(self._java_model, method)().toJavaRDD()
        return self._to_list(rdd)

# Returns as a list (false positive rate, true positive rate)
preds = test_pred.select(label_col,'probability').rdd.map(lambda row: (float(row['probability'][1]), float(row[label_col])))
points = CurveMetrics(preds).get_curve('roc')

x_val = [x[0] for x in points]
y_val = [x[1] for x in points]

In [0]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx,array[idx]
  
def get_x_for_y_points(x_list, y_list, y_points = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]):
  x_y_list = []
  for y in y_points:
    y_idx, value = find_nearest(y_list, y)
    x = round(x_list[y_idx],5)
    x_y_list.append( (y,x) ) 
    
  return x_y_list

#Compute FPR at interval of TPR
recall_fpr_list = get_x_for_y_points(x_val,y_val)

In [0]:
evaluator = BinaryClassificationEvaluator()

In [0]:
mlflow.end_run()
mlflow.start_run()
  
mlflow.log_param("training period", "2019-06-01 to 2019-06-30")
mlflow.log_param("testing period", "2019-10-01 to 2020-10-31")

mlflow.log_param("num_features", features_size)

mlflow.log_param("hidden units of LSTM layers", 64)
mlflow.log_param("output_dim", 2)
mlflow.log_param("group col", "card_approx")

mlflow.log_param("num_time_steps", 2)
mlflow.log_param("units of output_layer_1", 16)
mlflow.log_param("activation of output_layer1", "relu")
mlflow.log_param("units of output_layer_2", 8)
mlflow.log_param("activation of output_layer2", "relu")
mlflow.log_param("units of output_layer_3", 1)
mlflow.log_param("activation of output_layer3", "sigmoid")

mlflow.log_param("norm_type", "batch")
mlflow.log_param("tunning", "early_stoping")
mlflow.log_param("monitor", "valid_auc")
mlflow.log_param("batch_size", 1000)
mlflow.log_param("EPOCHS", 200)
train_scorelabel=train_pred.select(col(label_col).alias("label"),"rawPrediction")
test_scorelabel=test_pred.select(col(label_col).alias("label"),"rawPrediction")

mlflow.log_metric("trainAreaUnderROC", evaluator.evaluate(train_scorelabel, {evaluator.metricName: "areaUnderROC"}), step=1)
mlflow.log_metric("trainAreaUnderPR", evaluator.evaluate(train_scorelabel, {evaluator.metricName: "areaUnderPR"}), step=1)
mlflow.log_metric("testAreaUnderROC", evaluator.evaluate(test_scorelabel, {evaluator.metricName: "areaUnderROC"}), step=1)
mlflow.log_metric("testAreaUnderPR", evaluator.evaluate(test_scorelabel, {evaluator.metricName: "areaUnderPR"}), step=1)
if customer != "none":
  mlflow.log_metric("unseen_TPR_forFPR01", unseen_tpr01, step=1)
  mlflow.log_metric("unseen_TPR_forFPR001", unseen_tpr001, step=1)
  mlflow.log_metric("unseen_TPR_forFPR0001", unseen_tpr0001, step=1)

mlflow.log_metric("TPR_forFPR01", tpr01, step=1)
mlflow.log_metric("TPR_forFPR001", tpr001, step=1)
mlflow.log_metric("TPR_forFPR0001", tpr0001, step=1)
mlflow.log_metric("TPRdecr_forFPR01", perfdecr_01, step=1)
mlflow.log_metric("TPRdecr_forFPR001", perfdecr_001, step=1)
mlflow.log_metric("TPRdecr_forFPR0001", perfdecr_0001, step=1)  

mlflow.log_metric("MCC", max_mcc_tuple[0], step=1)
mlflow.log_metric("MCC threshold", max_mcc_tuple[1], step=1)
mlflow.log_metric("MCC threshold precision", max_mcc_precision , step=1)
mlflow.log_metric("MCC threshold recall", max_mcc_recall, step=1)

mlflow.log_metric("MCC TP", max_mcc_tp, step=1)
mlflow.log_metric("MCC FP", max_mcc_fp, step=1)
mlflow.log_metric("MCC TN", max_mcc_tn, step=1)
mlflow.log_metric("MCC FN", max_mcc_fn, step=1)

mlflow.log_metric("Recall 10 pct", recall_fpr_list[0][1], step=1)
mlflow.log_metric("Recall 20 pct", recall_fpr_list[1][1], step=1)
mlflow.log_metric("Recall 30 pct", recall_fpr_list[2][1], step=1)
mlflow.log_metric("Recall 40 pct", recall_fpr_list[3][1], step=1)
mlflow.log_metric("Recall 50 pct", recall_fpr_list[4][1], step=1)
mlflow.log_metric("Recall 60 pct", recall_fpr_list[5][1], step=1)
mlflow.log_metric("Recall 70 pct", recall_fpr_list[6][1], step=1)
mlflow.log_metric("Recall 80 pct", recall_fpr_list[7][1], step=1)
mlflow.log_metric("Recall 90 pct", recall_fpr_list[8][1], step=1)

mlflow.log_metric("Green TNR ", green_info[1] , step=1) #High TNR is higher TN
mlflow.log_metric("Green recall ", green_info[2], step=1) #Higher TPR means Lower FN
mlflow.log_metric("Green measure avg TNR and recall", green_info[0], step=1)
mlflow.log_metric("Green measure threshold", green_info[3], step=1)

start_week=start_test
end_week=datetime.strftime(datetime.strptime(start_week, "%Y-%m-%d") + timedelta(days=7), "%Y-%m-%d")
with open("performance_per_week_exp.txt", "w") as f:
  f.write("Train set ends at "+end_train+"\n")
  while end_week <= end_test:
    week_pred=handypredictions.filter(col("authtimestamp")>=start_week).filter(col("authtimestamp")<end_week)
    bcm = BinaryClassificationMetrics(week_pred, scoreCol='probability', labelCol=label_col)
    predbythreshold_week=bcm.getMetricsByThreshold()
    fpr01,tpr01,threshold01 = reusable_functions.getscoreswithfpr(predbythreshold_week,0.1)
    fpr001,tpr001,threshold001 = reusable_functions.getscoreswithfpr(predbythreshold_week,0.01)
    fpr0001,tpr0001,threshold0001 = reusable_functions.getscoreswithfpr(predbythreshold_week,0.001)
    f.write(start_week+" - "+end_week+", 10%: "+str(round(tpr01,3))+", 1%: "+ str(round(tpr001,3))+", 0.1%: "+str(round(tpr0001,3))+"\n")
    start_week=datetime.strftime(datetime.strptime(start_week, "%Y-%m-%d") + timedelta(days=7), "%Y-%m-%d")
    end_week=datetime.strftime(datetime.strptime(end_week, "%Y-%m-%d") + timedelta(days=7), "%Y-%m-%d")
mlflow.log_artifact("performance_per_week_exp.txt")